In [16]:
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t, norm

## Generating the Data

In [21]:
#================= Generating G matrix
rows = 40
cols = 3
t = np.linspace(0,4,40)
m_true = np.array([0.81,16.21,-9.81]).T
G  = np.zeros((rows,cols))

G[:,0] = 1
G[:,1] = t
G[:,2] = 0.5*t**2

In [22]:
#================= Computing d_true
d_true = G@m_true

#================= Generating the noisy data d
d = d_true + np.random.normal(0,2,40)
d

array([  1.97119839,   1.78011402,   6.42008764,   6.76565312,
         6.56332652,   9.11374884,   9.20605149,  10.92657712,
        11.76510148,  11.89243067,  13.04745299,  15.58807168,
        13.34160046,  17.52684341,  13.2352184 ,  15.05937047,
        13.27997893,  12.81816131,  15.89696322,  16.36221047,
        13.24459734,  12.78502101,  13.73909975,  12.149714  ,
        12.51079321,   9.86412666,   9.93581577,  10.19904478,
         6.72446248,   6.98948354,   0.5729276 ,   0.98039081,
        -0.32558916,  -2.50220313,  -2.15963581,  -6.26807702,
        -2.46960621,  -8.47274606, -11.93017188, -13.62406551])

## Q1

In [23]:
#================= Computing chi-square observed 
data_error = d - d_true
noise = 2*np.ones(40).T
chisqure_observed = np.linalg.norm((d_true-d)/noise)**2
chisqure_observed

#================= Computing the p-value 
m = 40
n = 3
dof = m - n 
#================= p-value
p_value = 1 - stats.chi2.cdf(chisqure_observed, dof)

print("P-value:", p_value)

P-value: 0.9008408013642508


This indicates that the fitted model is consistent with the assumptions on the modeling and data uncertainty.

## Q2

In [5]:
#================= least squares solution (m_L2)
m_L2 = np.linalg.lstsq(G,d,rcond=None)[0]

#================= Covariance matrix 
GG = G.T@G
cov_m_L2 = (2**2)*np.linalg.inv(GG)
cov_m_L2

array([[ 0.81567944, -0.80513937,  0.33122822],
       [-0.80513937,  1.09117573, -0.50991713],
       [ 0.33122822, -0.50991713,  0.25495857]])

In [6]:
#================= 95% Confindence Inveterval
def CI_95(m_L2, cov_m_L2):
    standard_error = np.sqrt(np.diag(cov_m_L2))
    lower = m_L2 - 1.96*standard_error
    upper = m_L2 + 1.96*standard_error
    return lower, upper

lower, upper = CI_95(m_L2, cov_m_L2)
print("Lower bounds:", lower)
print("Upper bounds:", upper)

Lower bounds: [  0.05652724  12.85449764 -10.30767821]
Upper bounds: [ 3.5968742  16.94930444 -8.32833607]


## Q3

In [7]:
from scipy.stats import t, norm
#================= computing sigma squared
simma_squared = (data_error.T@data_error)/(m-n)
simma_squared

#================= critical value
critical_value = t.ppf(1 - 0.05/2,m-n)
critical_value

2.0261924630291093

In [8]:
#================= 95% Confindence Inveterval
def CI_95_(m_L2, cov_m_L2, critical_value):
    standard_error = np.sqrt(np.diag(cov_m_L2))
    lower_ = m_L2 - critical_value*standard_error
    upper_ = m_L2 + critical_value*standard_error
    return lower_, upper_

lower_, upper_ = CI_95_(m_L2, cov_m_L2, critical_value)
print("Lower bounds:", lower_)
print("Upper bounds:", upper_)

Lower bounds: [-3.25446443e-03  1.27853534e+01 -1.03411011e+01]
Upper bounds: [ 3.6566559  17.01844867 -8.29491323]
